Pobieranie wiadomości z niebezpiecznik.pl Zapis do pliku ./data/niebezpiecznik_{timestamp}.txt

In [1]:
# ! pip install bs4
# ! pip install requests
# ! pip install tqdm
# ! pip install pandas
# ! pip install pyarrow
# ! pip install qgrid
# ! pip install blaze
# ! pip install ipynbname

In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm, trange
from datetime import datetime, timezone
from common.webarticle import WebArticle
from common.pl_helper import PolishLocaleHelper
import pandas as pd
import logging
import ipynbname

In [3]:
# settings:
portal_name = 'niebezpiecznik'
website = 'https://niebezpiecznik.pl'
pagination_attributes = '/page/'
no_articles_to_scrap = 300
data_dir = '../data'
logging.basicConfig(
    filename=f'../logs/{ipynbname.name()}.ipynb.log',
    encoding='utf-8',
    level=logging.DEBUG
)
ignored_paragraph_starts = []

Sprawdźmy plik robots.txt, w czasie pisania tego kodu pozwalał na pobieranie plików z portalu niebezpiecznik

In [4]:
resp = requests.get('https://www.niebezpiecznik.pl/robots.txt')
print(resp.text)

Sitemap: https://niebezpiecznik.pl/sitemap.xml




In [5]:
def check_if_ignored_paragraph(text, ignored_paragraphs, operation='startswith'):

    for ignored_paragraph in ignored_paragraphs:
        if operation == 'startswith':
            if text.startswith(ignored_paragraph):
                return True
        else:
            logging.debug(f'check_if_ignored: operation {operation} not implemented')
    return False


In [6]:
def scrap_article_text_niebezpiecznik(link):
    logging.info('scrap_article_text_niebezpiecznik:')
    logging.info(f'article link to scrap: {link}')

    page = requests.get(link)
    if page.status_code != 200:
        logging.debug(f'received status code: {page.status_code}')
        return '', ''

    soup = BeautifulSoup(page.content, 'html.parser')
    paragraphs = soup.find('div', class_='entry').find_all('p')

    text = ''
    lead_text_int = '' # there is no internal lead text in Niebezpiecznik posts

    for paragraph in paragraphs:
        paragraph_text = paragraph.get_text().strip()
        if check_if_ignored_paragraph(paragraph_text, ignored_paragraph_starts):
            continue
        text += paragraph_text + ' '

    #13:27\n27/6/2019
    time_string = soup.find('div', class_='post').find('div', class_='date').get_text().strip()
    when_published = datetime.strptime(time_string, "%H:%M\n%d/%m/%Y")

    title_int = soup.find('div', class_='title').find('h2').get_text().strip()

    author_string = soup.find('div', class_='title').find('div', class_='postmeta').get_text(strip=True)
    author = author_string.split('|')[0].replace('Autor: ','')
    source = ''

    return text, when_published, title_int, lead_text_int, author, source

In [7]:
def scrap_articles_niebezpiecznik(website):
    logging.info('scrap_articles_niebezpiecznik:')
    logging.info(f'page with articles to scrap: {website}')

    result = list()
    page = requests.get(website)
    if page.status_code != 200:
        logging.debug(f'received status code: {page.status_code}')
        return result

    soup = BeautifulSoup(page.content, 'html.parser')
    page_content = soup.find('div',id='main').find_all('div',class_='post')

    print(f'articles to scrap: {len(page_content)}')

    for article in page_content:

        logging.info(f"article scrapping started: {article}")
        title = article.find('div',class_='title').find('h2').get_text().strip()
        lead_text = article.find('div', class_='entry').get_text().strip()
        link = article.find('div', class_='title').find('a', href=True, rel='bookmark')['href']
        webart = WebArticle(title, lead_text, link, *scrap_article_text_niebezpiecznik(link),portal_name)
        logging.info(f'webarticle scrapped: {webart}')
        result.append(webart)

    return result

In [8]:
no_scrapped_articles = 0
page_no=1
scrapped_articles = list()

logging.info(f'no_articles_to_scrap: {no_articles_to_scrap}')

pbar = tqdm(total=no_articles_to_scrap)
while no_scrapped_articles < no_articles_to_scrap:
    logging.info('next page: '+website+pagination_attributes+str(page_no))
    new_articles = scrap_articles_niebezpiecznik(website+pagination_attributes+str(page_no))
    page_size = len(new_articles)
    logging.info(f'new_articles: {page_size}')
    if no_scrapped_articles + page_size > no_articles_to_scrap:
        page_size = no_articles_to_scrap - no_scrapped_articles
    scrapped_articles.extend(new_articles[:page_size])
    logging.info(f'scrapped new articles: {page_size}')
    no_scrapped_articles += page_size
    pbar.update(page_size)
    logging.info(f'no_scrapped_articles (total): {no_scrapped_articles}')
    page_no += 1
    if len(scrapped_articles) <= 0:
        logging.debug(f'0 articles scrapped!')
        break
pbar.close()

  0%|          | 0/300 [00:00<?, ?it/s]

articles to scrap: 20


  7%|▋         | 20/300 [00:11<02:45,  1.69it/s]

articles to scrap: 20


 13%|█▎        | 40/300 [00:21<02:19,  1.87it/s]

...

 87%|████████▋ | 260/300 [02:12<00:20,  1.93it/s]

articles to scrap: 20


 93%|█████████▎| 280/300 [02:20<00:09,  2.02it/s]

articles to scrap: 20


100%|██████████| 300/300 [02:29<00:00,  2.01it/s]


In [9]:
for i in trange(len(scrapped_articles)):
    # print(f'{i}: {scrapped_articles[i].lead_text}')
    print(f'{i}: {len(scrapped_articles[i].text)}')

100%|██████████| 300/300 [00:00<00:00, 33228.35it/s]

0: 5395
1: 1883
2: 8410
3: 1935
4: 5832
5: 4992
6: 1541
7: 1269
8: 4943
9: 4050
10: 5118
11: 11249
12: 10131
13: 2363
14: 740
15: 2452
16: 16551
17: 6441
18: 2494
19: 500
20: 4343
21: 2866
22: 4337
23: 986
24: 2641
25: 19112
26: 2847
27: 1388
28: 1281
29: 10125
30: 4922
31: 5996
32: 3556
33: 7674
34: 2076
35: 18395
36: 2019
37: 6570
38: 4542
39: 2314
40: 1346
41: 2594
42: 3899
43: 2327
44: 2305
45: 3168
46: 3504
47: 8791
48: 5006
49: 4270
50: 1893
51: 1866
52: 4841
53: 5517
54: 5552
55: 8414
56: 4425
57: 1970
58: 6883
59: 3122
60: 15717
61: 3178
62: 1403
63: 9015
64: 3278
65: 984
66: 921
67: 12812
68: 1960
69: 5589
70: 6194
71: 3846
72: 1463
73: 4101
74: 2264
75: 1383
76: 7661
77: 1223
78: 9460
79: 1054
80: 11167
81: 8384
82: 18599
83: 5439
84: 2178
85: 2407
86: 14236
87: 8756
88: 2633
89: 2629
90: 3412
91: 6609
92: 3893
93: 1849
94: 5682
95: 2100
96: 8605
97: 7118
98: 2078
99: 2482
100: 17881
101: 1449
102: 3925
103: 5916
104: 7035
105: 4331
106: 3345
107: 1885
108: 3876
109: 2461
110

In [10]:
niebezpiecznik_dataframe = pd.DataFrame(article.__dict__ for article in scrapped_articles)
niebezpiecznik_dataframe

,title,title_int,lead_text,lead_text_int,link,text,when_published,author,source,portal
0,📌 OSINT (Biały Wywiad) czyli techniki pozyskiw...,📌 OSINT (Biały Wywiad) czyli techniki pozyskiw...,"Na konferencji CONFidence, w trakcie swojego w...",,https://niebezpiecznik.pl/post/osint-bialy-wyw...,"Na konferencji CONFidence, w trakcie swojego w...",2019-06-27 13:27:00,redakcja,,niebezpiecznik
1,Prokuratura szuka poszkodowanych w scamie na KGHM,Prokuratura szuka poszkodowanych w scamie na KGHM,"Pół roku temu opisaliśmy przekręt ""na Orlen"". ...",,https://niebezpiecznik.pl/post/prokuratura-szu...,Pół roku temu opisaliśmy przekręt “na Orlen”....,2022-01-17 13:26:00,redakcja,,niebezpiecznik
2,Jak poważny jest “gigantyczny wyciek z wojska”?,Jak poważny jest “gigantyczny wyciek z wojska”?,"W piątek Onet poinformował o ""gigantycznym wyc...",,https://niebezpiecznik.pl/post/jak-powazny-jes...,W piątek Onet poinformował o “gigantycznym wy...,2022-01-16 19:10:00,redakcja,,niebezpiecznik
3,Ostrzeżenie dla polskich firm utrzymujących ko...,Ostrzeżenie dla polskich firm utrzymujących ko...,Na komputerach wielu organizacji non-profitowy...,,https://niebezpiecznik.pl/post/ostrzezenie-pol...,Na komputerach wielu organizacji non-profitow...,2022-01-16 09:58:00,Autor:Piotr Konieczny,,niebezpiecznik
4,[AKTUALIZACJA] Ktoś chciał nas wplątać w cyber...,[AKTUALIZACJA] Ktoś chciał nas wplątać w cyber...,Wczorajszy dzień obfitował w informacje dotycz...,,https://niebezpiecznik.pl/post/ktos-chcial-nas...,Wczorajszy dzień obfitował w informacje dotyc...,2022-01-15 11:49:00,Autor:Piotr Konieczny,,niebezpiecznik
...,...,...,...,...,...,...,...,...,...,...
295,"Uwaga na ""pożyczki przez internet"". 2 przykład...","Uwaga na ""pożyczki przez internet"". 2 przykład...",W czasach zarazy ludzie potrzebują pieniędzy. ...,,https://niebezpiecznik.pl/post/uwaga-na-pozycz...,W czasach zarazy ludzie potrzebują pieniędzy....,2020-06-01 19:55:00,Marcin Maj,,niebezpiecznik
296,Czy polskie firmy są gotowe na bezpieczną prac...,Czy polskie firmy są gotowe na bezpieczną prac...,Aktualna sytuacja na świecie wymusiła na wielu...,,https://niebezpiecznik.pl/post/czy-polskie-fir...,Aktualna sytuacja na świecie wymusiła na wiel...,2020-06-01 08:46:00,redakcja,,niebezpiecznik
297,Phishing i steganografia w interesującym ataku...,Phishing i steganografia w interesującym ataku...,Przestępcy bywają kreatywni czego dowodem są a...,,https://niebezpiecznik.pl/post/phishing-i-steg...,Przestępcy bywają kreatywni czego dowodem są ...,2020-05-29 12:03:00,Marcin Maj,,niebezpiecznik
298,Sklep Niebezpiecznika ruszył,Sklep Niebezpiecznika ruszył,Co to dużo pisać. Sporo osób przez lata męczył...,,https://niebezpiecznik.pl/post/sklep-niebezpie...,Co to dużo pisać. Sporo osób przez lata męczy...,2020-05-27 23:11:00,redakcja,,niebezpiecznik


In [11]:
timestamp = int((datetime.now(timezone.utc)).timestamp())

In [12]:
niebezpiecznik_dataframe.to_parquet(f'{data_dir}/{portal_name}_{timestamp}.parquet')
niebezpiecznik_dataframe.to_json(f'{data_dir}/{portal_name}_{timestamp}.json')

Krótki test - dane pierwszego artykułu

In [13]:
print(f'title:\n  {niebezpiecznik_dataframe.iloc[0]["title"]}')
print(f'lead_text:\n  {niebezpiecznik_dataframe.iloc[0]["lead_text"]}')
print(f'lead_text_int:\n  {niebezpiecznik_dataframe.iloc[0]["lead_text_int"]}')
print(f'link:\n  {niebezpiecznik_dataframe.iloc[0]["link"]}')
print(f'text:\n  {niebezpiecznik_dataframe.iloc[0]["text"]}')
print(f'when_published:\n  {niebezpiecznik_dataframe.iloc[0]["when_published"]}')

title:
  📌 OSINT (Biały Wywiad) czyli techniki pozyskiwania informacji o ludziach i firmach
lead_text:
  Na konferencji CONFidence, w trakcie swojego wykładu, Piotrek zapowiedział start nowego niebezpiecznikowego szkolenia poświęconego tematyce OSINT-u (ang. Open Source Intelligence), czyli technikom pozyskiwania informacji na temat ludzi i firm na podstawie ogólniedostępnych (co nie znaczy, że zawsze publicznie dostępnych w internecie) źródeł. Miło nam zakomunikować, że właśnie ruszył nabór na pierwsze z terminów tego szkolenia! Czytaj dalej »
124 komentarzy
lead_text_int:
  
link:
  https://niebezpiecznik.pl/post/osint-bialy-wywiad-czyli-techniki-pozyskiwania-informacji-o-ludziach-i-firmach/
text:
  Na konferencji CONFidence, w trakcie swojego wykładu, Piotrek zapowiedział start nowego niebezpiecznikowego szkolenia poświęconego tematyce OSINT-u (ang. Open Source Intelligence), czyli technikom pozyskiwania informacji na temat ludzi i firm na podstawie ogólniedostępnych (co nie znaczy,

In [14]:
print(f'title:\n  {niebezpiecznik_dataframe.iloc[99]["title"]}')
print(f'lead_text:\n  {niebezpiecznik_dataframe.iloc[99]["lead_text"]}')
print(f'link:\n  {niebezpiecznik_dataframe.iloc[99]["link"]}')
print(f'text:\n  {niebezpiecznik_dataframe.iloc[99]["text"]}')
print(f'when_published:\n  {niebezpiecznik_dataframe.iloc[99]["when_published"]}')

title:
  Webinar: Kontrola Urzędu Ochrony Danych Osobowych. Jak się przygotować? Co Ci grozi?
lead_text:
  Jeśli Twoja firma jeszcze nie zgłosiła żadnego naruszenia do UODO, to albo coś ukrywasz albo takie naruszenie przyjdzie Ci zgłosić do końca roku. Wycieki danych są obecnie na porządku dziennym i niestety, prędzej czy później dotkną każdej firmy... Oczywiście nie każde zgłoszenie wycieku do UODO zakończy się kontrolą, ale niektóre z nich mogą... 
Kontrola UODO to ... Czytaj dalej »
6 komentarzy
link:
  https://niebezpiecznik.pl/post/webinar-kontrola-urzedu-ochrony-danych-osobowych-uodo/
text:
   Jeśli Twoja firma jeszcze nie zgłosiła żadnego naruszenia do UODO, to albo coś ukrywasz albo takie naruszenie przyjdzie Ci zgłosić do końca roku. Wycieki danych są obecnie na porządku dziennym i niestety, prędzej czy później dotkną każdej firmy… Oczywiście nie każde zgłoszenie wycieku do UODO zakończy się kontrolą, ale niektóre z nich mogą…  Kontrola UODO to dla wielu stresująca sytuacja, d

In [15]:
print(f'title:\n  {niebezpiecznik_dataframe.iloc[16]["title"]}')
print(f'title_int:\n  {niebezpiecznik_dataframe.iloc[16]["title_int"]}')
print(f'lead_text:\n  {niebezpiecznik_dataframe.iloc[16]["lead_text"]}')
print(f'lead_text_int:\n  {niebezpiecznik_dataframe.iloc[16]["lead_text_int"]}')
print(f'link:\n  {niebezpiecznik_dataframe.iloc[16]["link"]}')
print(f'text:\n  {niebezpiecznik_dataframe.iloc[16]["text"]}')
print(f'when_published:\n  {niebezpiecznik_dataframe.iloc[16]["when_published"]}')
print(f'author:\n  {niebezpiecznik_dataframe.iloc[16]["author"]}')

title:
  Nie wykorzystałeś budżetu szkoleniowego? Kup vouchery Niebezpiecznika na 2022
title_int:
  Nie wykorzystałeś budżetu szkoleniowego? Kup vouchery Niebezpiecznika na 2022
lead_text:
  Za chwilę koniec roku, a od stycznia nowy ład i nowe, wyższe ceny. Jeśli chciałbyś wziąć udział w naszych szkoleniach po starych cenach, ale grudniowe terminy Ci nie pasują, zachęcamy do zakupu vouchera, który będziesz mógł wykorzystać w 2022 roku (faktura zostanie wystawiona z datą zakupu). Vouchery nie są imienne, więc nie musisz z niego ... Czytaj dalej »
1 komentarz
lead_text_int:
  
link:
  https://niebezpiecznik.pl/post/nie-wykorzystales-budzetu-szkoleniowego-kup-vouchery-niebezpiecznika-na-2022/
text:
   Za chwilę koniec roku, a od stycznia nowy ład i nowe, wyższe ceny. Jeśli chciałbyś wziąć udział w naszych szkoleniach po starych cenach, ale grudniowe terminy Ci nie pasują, zachęcamy do zakupu vouchera, który będziesz mógł wykorzystać w 2022 roku (faktura zostanie wystawiona z datą zakupu).